In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [5]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [9]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [11]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [13]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.0606 - loss: 6.5499
Epoch 2/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.1174 - loss: 5.5818
Epoch 3/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 46s 15ms/step - accuracy: 0.1449 - loss: 5.1280
Epoch 4/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 44s 15ms/step - accuracy: 0.1646 - loss: 4.7807
Epoch 5/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - accuracy: 0.1825 - loss: 4.4533
Epoch 6/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - accuracy: 0.2033 - loss: 4.1575
Epoch 7/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.2322 - loss: 3.8680
Epoch 8/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - accuracy: 0.2673 - loss: 3.5834
Epoch 9/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - accuracy: 0.3029 - loss: 3.3471
Epoch 10/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - accuracy: 0.3394 - loss: 3.1165
Epoch 11/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - accuracy: 0.3741 - loss: 2.91

In [21]:
seed_text = "What is"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
What is the name of
